<a href="https://colab.research.google.com/github/hislam000/CSC-599.70-Course-Project/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Intro to Data Science, Fall 2019 @ CCNY
# CSC-599.70-Course-Project
# Final-Project
# Team Member & Name: RentAdvisor (3-Member)

# HASIBUL ISLAM
# DZHONIBEK PARMANKULOV
# ABDUR RAFEY

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [0]:
g'dfnh